In [37]:
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!!unzip -o annotations_trainval2017.zip

--2022-01-18 12:27:36--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.138.129
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.138.129|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



['Archive:  annotations_trainval2017.zip',
 '  inflating: annotations/instances_train2017.json  ',
 '  inflating: annotations/instances_val2017.json  ',
 '  inflating: annotations/captions_train2017.json  ',
 '  inflating: annotations/captions_val2017.json  ',
 '  inflating: annotations/person_keypoints_train2017.json  ',
 '  inflating: annotations/person_keypoints_val2017.json  ']

In [45]:
import json
import pandas as pd

img_to_check=[]
KEYWORDS=['sand[ y\.]','grass[ y\.]','snow[ y\.]','wind[ y\.]','mountain','valley','waterfall','forest','city','hill','valley','sky','cliff',' ridge','mist[ y\.]','cloud[ y\.]','lake','countryside','sea','ocean']
with open('./annotations/captions_val2017.json') as json_file:
    data = json.load(json_file)

df= pd.DataFrame(data['annotations'])
df2=df[df['caption'].str.contains('|'.join(KEYWORDS))]
df2

img_to_check.extend([(k['coco_url'],k['id']) for k in data['images'] if k['id'] in set(df2['image_id'])])


with open('./annotations/captions_train2017.json') as json_file:
    data = json.load(json_file)

df= pd.DataFrame(data['annotations'])
df3=df[df['caption'].str.contains('|'.join(KEYWORDS))]
df3

img_to_check2=((k['coco_url'],k['id']) for k in data['images'] if k['id'] in set(df3['image_id']))

,image_id,id,caption
4,106140,101,There is a GOL plane taking off in a partly cl...
13,301837,391,"A long empty, minimal modern skylit home kitchen."
25,459912,748,Rows of motor bikes and helmets in a city
34,226419,939,A random plane in the sky flying alone
35,226419,945,An airplane flying high in the blue sky.
...,...,...,...
591574,87685,823276,A clock tower lit up at night time in a city.
591580,342919,823444,A blue teddy bear sitting behind a snow globe ...
591598,500705,824378,Various market booths are set up in a city.
591625,206416,825543,A brick clock tower is visible in front of a c...


In [ ]:
import random
import time
from IPython.display import Image, display, clear_output
from ipywidgets import widgets

queue=img_to_check
result=[]
done=[]
def choose():
    
    if queue:
      (im,imgid)=queue.pop()
      
      if imgid in done:
        choose()
      else:
      
        button1 = widgets.Button(description = "keep")
        button2 = widgets.Button(description = "drop")

        container = widgets.HBox(children=[button1,button2])
        


        def on_button1_clicked(b):
            done.append(imgid)
            result.append(imgid)
            container.close()
            clear_output()
            choose()

        def on_button2_clicked(b):
            done.append(imgid)
            container.close()
            clear_output()
            choose()


        button1.on_click(on_button1_clicked)
        button2.on_click(on_button2_clicked)
        display(Image(im))
        display(container)

choose() # initializes the first choice
print(result)

In [ ]:
!pip install fiftyone
# !pip uninstall --yes opencv-python-headless==4.5.5.62
# !pip install opencv-python-headless==4.5.2.52

In [ ]:
import fiftyone.zoo as foz
import fiftyone as fo


dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    image_ids=set(df2['image_id'])
)
session = fo.launch_app(dataset)

dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    image_ids=set(df3['image_id'])
)
session = fo.launch_app(dataset)